In [3]:
def predict_missing(data, over, bowler_type="fast"):
    # SPEED
    if data["speed"] is None:
        if over <= 6:
            data["speed"] = 140
        elif over <= 15:
            data["speed"] = 135
        else:
            data["speed"] = 145

    # LENGTH
    if data["length"] is None:
        if over <= 6:
            data["length"] = "full"
        elif over <= 15:
            data["length"] = "good"
        else:
            data["length"] = "yorker"

    # LINE
    if data["line"] is None:
        data["line"] = "off"

    # SWING
    if data["swing"] is None:
        if over <= 6 and bowler_type == "fast":
            data["swing"] = "outswing"
        else:
            data["swing"] = "none"

    return data


In [4]:
import re

# ---------------- SPEED ----------------
def extract_speed(text):
    match = re.search(r'(\d{2,3})\s?(km\/h|kph|clicks)', text.lower())
    return int(match.group(1)) if match else None


# ---------------- LENGTH ----------------
LENGTH_MAP = {
    "yorker": "yorker",
    "full": "full",
    "good length": "good",
    "back of a length": "short",
    "short": "short"
}

def extract_length(text):
    for k, v in LENGTH_MAP.items():
        if k in text.lower():
            return v
    return None


# ---------------- LINE ----------------
LINE_MAP = {
    "outside off": "off",
    "off stump": "off",
    "middle": "middle",
    "leg stump": "leg",
    "down leg": "leg",
    "wide": "wide",
    "straight": "middle"
}

def extract_line(text):
    for k, v in LINE_MAP.items():
        if k in text.lower():
            return v
    return None


# ---------------- SWING ----------------
SWING_MAP = {
    "shapes away": "outswing",
    "swinging away": "outswing",
    "seams away": "outswing",
    "shapes in": "inswing",
    "jags back": "inswing",
    "straightens": "none"
}

def extract_swing(text):
    for k, v in SWING_MAP.items():
        if k in text.lower():
            return v
    return None


In [ ]:
def process_ball(commentary, over):
    data = {
        "speed": extract_speed(commentary),
        "length": extract_length(commentary),
        "line": extract_line(commentary),
        "swing": extract_swing(commentary)
    }

    data = predict_missing(data, over)
    return data


In [ ]:
def unity_mapping(data):
    return {
        "speedForce": data["speed"] * 0.22,
        "lineX": {
            "off": 0.3,
            "middle": 0.0,
            "leg": -0.3,
            "wide": 0.6
        }[data["line"]],
        "lengthZ": {
            "yorker": 5.5,
            "full": 7.0,
            "good": 9.5,
            "short": 13.0
        }[data["length"]],
        "swingForce": {
            "outswing": 0.4,
            "inswing": -0.4,
            "none": 0.0
        }[data["swing"]]
    }
